In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('sentence-transformers/stsb-roberta-base-v2')

data = pd.read_json("data/climate-fever.jsonl", lines=True)
data

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,claim_id,claim,claim_label,evidences
0,0,Global warming is driving polar bears toward e...,SUPPORTS,[{'evidence_id': 'Extinction risk from global ...
1,5,The sun has gone into ‘lockdown’ which could c...,SUPPORTS,"[{'evidence_id': 'Famine:386', 'evidence_label..."
2,6,The polar bear population has been growing.,REFUTES,"[{'evidence_id': 'Polar bear:1332', 'evidence_..."
3,9,Ironic' study finds more CO2 has slightly cool...,REFUTES,"[{'evidence_id': 'Atmosphere of Mars:131', 'ev..."
4,10,Human additions of CO2 are in the margin of er...,REFUTES,[{'evidence_id': 'Carbon dioxide in Earth's at...
...,...,...,...,...
1530,3125,About 60% of the warming observed from 1970 to...,NOT_ENOUGH_INFO,"[{'evidence_id': 'Climate variability:103', 'e..."
1531,3127,"""Skeptics hope that Postma’s alternative therm...",NOT_ENOUGH_INFO,"[{'evidence_id': 'Meteorology:4', 'evidence_la..."
1532,3130,"""There are other possible causes for climate c...",SUPPORTS,[{'evidence_id': 'Attribution of recent climat...
1533,3131,We don't need a high heat flow - just a high t...,NOT_ENOUGH_INFO,"[{'evidence_id': 'Earth:161', 'evidence_label'..."


In [3]:
fact = []

for evidences in data["evidences"]:
    for evidence in evidences:
        fact.append(evidence["evidence"])

fact = list(set(fact))

embeddings = model.encode(fact,show_progress_bar=True)

Batches: 100%|██████████| 164/164 [05:02<00:00,  1.85s/it]


In [4]:
import json

df = pd.DataFrame(fact)

embds = []

for embd in embeddings:
    embds.append(json.dumps(embd.tolist()))

df.insert(1,"embeding",embds)
df.to_csv("data/climate_fever_evidence_embedding.csv",header=False,index=False)

In [7]:
import json

pairs = []
claim = []
evidence = []
voter1 = []
voter2 = []
voter3 = []
voter4 = []
voter5 = []

for claimeuh in data["claim"]:
    for i in range(5):
        claim.append(claimeuh)

for evidenceuhs in data["evidences"]:
    for evidenceuh in evidenceuhs:
        evidence.append(evidenceuh["evidence"])
        voter1.append(evidenceuh["votes"][0])
        voter2.append(evidenceuh["votes"][1])
        voter3.append(evidenceuh["votes"][2])
        voter4.append(evidenceuh["votes"][3])
        voter5.append(evidenceuh["votes"][4])


for pair in range(len(claim)):
    pairs.append(json.dumps([claim[pair], evidence[pair]]))


df_voter_1 = pd.DataFrame(pairs,columns=["pair"])
df_voter_1.insert(1,"vote",voter1)
df_voter_2 = pd.DataFrame(pairs,columns=["pair"])
df_voter_2.insert(1,"vote",voter2)
df_voter_3 = pd.DataFrame(pairs,columns=["pair"])
df_voter_3.insert(1,"vote",voter3)
df_voter_4 = pd.DataFrame(pairs,columns=["pair"])
df_voter_4.insert(1,"vote",voter4)
df_voter_5 = pd.DataFrame(pairs,columns=["pair"])
df_voter_5.insert(1,"vote",voter5)


df_voter_1['vote'] = df_voter_1['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
df_voter_2['vote'] = df_voter_2['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
df_voter_3['vote'] = df_voter_3['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
df_voter_4['vote'] = df_voter_4['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
df_voter_5['vote'] = df_voter_5['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])

df_voter_1.to_csv("data/climate_fever_voter_1.csv",header=True,index=False)
df_voter_2.to_csv("data/climate_fever_voter_2.csv",header=True,index=False)
df_voter_3.to_csv("data/climate_fever_voter_4.csv",header=True,index=False)
df_voter_4.to_csv("data/climate_fever_voter_5.csv",header=True,index=False)
df_voter_5.to_csv("data/climate_fever_voter_3.csv",header=True,index=False)

C:\Users\enzol\AppData\Local\Temp\ipykernel_8000\2674997119.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_voter_1['vote'] = df_voter_1['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
C:\Users\enzol\AppData\Local\Temp\ipykernel_8000\2674997119.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_voter_2['vote'] = df_voter_2['vote'].replace(['NOT_ENOUGH_INFO', None, 'REFUTES', 'SUPPORTS'],[0,3,2,1])
C:\Users\enzol\AppData\Local\Temp\ipykernel_8000\2674997119.py:44: FutureWarning: Downcasting 

In [10]:
import json

votes = []

for evidenceuhs in data["evidences"]:
    votes.append(json.dumps([json.dumps(evidenceuhs[0]["votes"]),
                             json.dumps(evidenceuhs[1]["votes"]),
                             json.dumps(evidenceuhs[2]["votes"]),
                             json.dumps(evidenceuhs[3]["votes"]),
                             json.dumps(evidenceuhs[4]["votes"])]))


df_label = pd.DataFrame(data["claim_label"])
df_label.insert(1,"votes",votes)

df_label['claim_label'] = df_label['claim_label'].replace(['NOT_ENOUGH_INFO', 'DISPUTED', 'REFUTES', 'SUPPORTS'],[0,3,2,1])

df_label.to_csv("data/climate_fever_labels.csv",header=True,index=False)

C:\Users\enzol\AppData\Local\Temp\ipykernel_8000\3948040976.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_label['claim_label'] = df_label['claim_label'].replace(['NOT_ENOUGH_INFO', 'DISPUTED', 'REFUTES', 'SUPPORTS'],[0,3,2,1])
